<a href="https://colab.research.google.com/github/kanyujidebug/kenkyuyo/blob/main/time_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install changefinder
!pip install sklearn
!pip install tslearn

     |████████████████████████████████| 154 kB 6.4 MB/s 
  Created wheel for changefinder: filename=changefinder-0.3-py3-none-any.whl size=3841 sha256=504ea486ec5a6ca400fba02e53ca25b3e89674654f3d148f9bef841489c28645
  Stored in directory: /root/.cache/pip/wheels/48/8f/4c/ba040e4cad3dbf0409c5c38ac9d3f92f947ea490412cdf3400
Successfully built changefinder


In [6]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from matplotlib import pylab as plt
import seaborn as sns
%matplotlib inline

import changefinder
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.utils import to_time_series_dataset

sns.set()
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
#マスキングされた年次毎件数データをアップロード
df=pd.read_csv('/content/drive/My Drive/merge_df_year.csv',index_col=0)

In [ ]:
df_year_test.index = pd.to_datetime(df_year_test.index,format='%Y%m')
df_year_test=df_year_test.loc[:'2021-09-01']

In [ ]:
df_year_test.plot()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
data=df_year_test.to_numpy()

cf = changefinder.ChangeFinder(r=0.07, order=1, smooth=4)

ret = []
for i in data:
    score = cf.update(i)
    ret.append(score)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df_year_test.index,ret)
ax2 = ax.twinx()
ax2.plot(df_year_test.index,data,'r')
ax.set_xlabel('year',size=12)
ax.set_ylabel('Rate',size=12)
ax2.set_ylabel('number of cases',size=12)
plt.savefig('/content/drive/My Drive/clinical_df_cancer_change.png')
plt.show()

plt.close()

In [ ]:
df_test=df.loc['200601':'202108','diabets']
df_test.index = pd.to_datetime(df_year_test_2.index,format='%Y%m')

In [ ]:
import matplotlib.pyplot as plt
import changefinder
import numpy as np
data=df_year_test_2.to_numpy()

cf = changefinder.ChangeFinder(r=0.07, order=1, smooth=10)

ret = []
for i in data:
    score = cf.update(i)
    ret.append(score)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df_year_test_2.index,ret)
ax2 = ax.twinx()
ax2.plot(df_year_test_2.index,data,'r')
ax.set_xlabel('year',size=12)
ax.set_ylabel('Rate',size=12)
ax2.set_ylabel('number of cases',size=12)
plt.savefig('/content/drive/My Drive/clinical_df_cancer_change_2.png')
plt.show()


plt.close()

In [ ]:
df.index = pd.to_datetime(df.index,format='%Y%m')
dfT = df.T

In [ ]:
df_roll=df.rolling(6).mean().round(1)
df_roll_all=df_roll.loc['2018-04-01':'2021-09-01','all']
df_roll=df_roll.drop('all',axis=1)

In [ ]:
df_year_test_2=df_roll.loc['2018-04-01':'2021-09-01'].div(df_roll_all,axis=0)
df_year_test_2=df_year_test_2.drop('Cancer',axis=1)

In [ ]:
list_=df_year_test_2.loc[:,df_year_test_2.mean()<0.01]
columns_list=list_.columns

In [ ]:
def tsclusteringN(ts_data, names):

    metric = 'dtw'
    n_clusters = [n for n in range(2, 6)]
    for n in n_clusters:
        print('クラスター数 =', n)
        km = TimeSeriesKMeans(n_clusters=n, metric=metric, verbose=False, random_state=1).fit(ts_data)
        print('クラスタリング結果 =', km.labels_)
        print('シルエット値 =', silhouette_score(ts_data, km.labels_, metric=metric))
        print()
 
def main():
    tsclusteringN(df_year_test_2.drop(columns_list,axis=1).loc['2018-04-01':'2021-09-01'].values.transpose(), df_year_test_2.drop(columns_list,axis=1).loc['2018-04-01':'2021-09-01'].columns)
 
if __name__ == '__main__':
    main()

クラスター数 = 2
クラスタリング結果 = [0 1 1 1 0 0 1]
シルエット値 = 0.33693767540165054

クラスター数 = 3
クラスタリング結果 = [1 2 0 0 0 0 2]
シルエット値 = 0.5890770556267251

クラスター数 = 4
クラスタリング結果 = [1 2 3 3 0 0 2]
シルエット値 = 0.760306136975082

クラスター数 = 5
クラスタリング結果 = [1 4 3 3 0 0 2]
シルエット値 = 0.5714285714285714



In [ ]:
# ターミナル対応
import matplotlib
matplotlib.use('Agg')
 
def plot_clustering(km, ts_dataset, names, n_clusters):
    # クラスタリングの結果をプロット
 
    # クラスターごとの中心をプロット
    for i, c in enumerate(km.cluster_centers_):
        plt.plot(c.T[0], label=i)
        plt.xticks(c.T[0],df_year_test_2.index)
 
    plt.legend(bbox_to_anchor=(1, 1),loc='upper left')
    
    plt.show()
    plt.savefig('ts_clust_center.png',bbox_inches="tight")
    plt.clf()
 
    # クラスターごとのプロット
    for i in range(n_clusters):
        for label, d, t in zip(km.labels_, ts_dataset, names):
            if label == i:
                plt.plot(d, label=t)
 
        plt.legend(bbox_to_anchor=(1, 1),loc='upper left')
        plt.show()
        plt.savefig('ts_labeled{}.png'.format(i),bbox_inches="tight")
        plt.clf()
        print()
 
def tsclustering(ts_data, names):
    # 正規化
    #ts_dataset = TimeSeriesScalerMinMax().fit_transform(ts_data)
 
    n_clusters = 4
    metric = 'dtw'
 
    # metricが「DTW」か「softdtw」なら異なるデータ数の時系列データでもOK
    km = TimeSeriesKMeans(n_clusters=n_clusters, metric=metric, verbose=False, random_state=1).fit(ts_data)
 
    # クラスタリングの結果
    print('クラスタリング結果 =', km.labels_)
 
    plot_clustering(km, ts_data, names, n_clusters)
 
def main():
    #tsclusteringN(df.values.transpose(), df.columns)
    tsclustering(df_year_test_2.drop(columns_list,axis=1).loc['2018-04-01':'2021-09-01'].values.transpose(), df_year_test_2.drop(columns_list,axis=1).loc['2018-04-01':'2021-09-01'].columns)
 
if __name__ == '__main__':
    main()

クラスタリング結果 = [1 2 3 3 0 0 2]




